In [12]:
import json
import networkx as nx
import matplotlib.pyplot as plt
from collections import deque
import heapq

class GraphEngine:
    def __init__(self, json_file):
        # Importation du fichier JSON fourni par l'utilisateur 
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        self.adj = data["villes"]
        
        # Construction du graphe pour NetworkX et les calculs [cite: 9, 10]
        self.G = nx.Graph()
        for v, neighbors in self.adj.items():
            for n, w in neighbors:
                self.G.add_edge(v, n, weight=w)
        
        # Coordonnées pour la visualisation (Disposition fixe pour le rapport)
        self.pos = {
            "Rennes": (0, 3), "Caen": (1, 4), "Lille": (4, 5),
            "Paris": (2, 3), "Nantes": (0, 2), "Bordeaux": (0, 0),
            "Dijon": (4, 2), "Nancy": (5, 4), "Lyon": (4, 1), "Grenoble": (5, 0)
        }

    def tracer_graphe(self, titre, path=None, edges=None):
        plt.figure(figsize=(10, 6))
        nx.draw(self.G, self.pos, with_labels=True, node_color='lightgray', edge_color='gray', style='dotted', node_size=1200)
        
        if path: # Surlignage de nodes (BFS/DFS/Dijkstra)
            path_edges = [(path[i], path[i+1]) for i in range(len(path)-1)]
            nx.draw_networkx_edges(self.G, self.pos, edgelist=path_edges, edge_color='red', width=3)
            nx.draw_networkx_nodes(self.G, self.pos, nodelist=path, node_color='orange')
        
        if edges: # Surlignage d'arêtes (MST)
            nx.draw_networkx_edges(self.G, self.pos, edgelist=edges, edge_color='blue', width=3)

        labels = nx.get_edge_attributes(self.G, 'weight')
        nx.draw_networkx_edge_labels(self.G, self.pos, edge_labels=labels)
        plt.title(titre)
        plt.show()

    # --- Algorithmes ---
    def bfs(self, start): # [cite: 52, 67]
        visited, queue, order = {start}, deque([start]), []
        while queue:
            u = queue.popleft()
            order.append(u)
            for v, _ in self.adj.get(u, []):
                if v not in visited:
                    visited.add(v)
                    queue.append(v)
        return order

    def dfs(self, start, visited=None, order=None): # [cite: 52, 67]
        if visited is None: visited, order = set(), []
        visited.add(start)
        order.append(start)
        for v, _ in self.adj.get(start, []):
            if v not in visited: self.dfs(v, visited, order)
        return order

    def kruskal(self): # [cite: 53, 77]
        parent = {n: n for n in self.G.nodes()}
        def find(i):
            if parent[i] == i: return i
            return find(parent[i])
        edges = sorted(self.G.edges(data=True), key=lambda x: x[2]['weight'])
        mst, cost = [], 0
        for u, v, d in edges:
            root_u, root_v = find(u), find(v)
            if root_u != root_v:
                parent[root_u] = root_v
                mst.append((u, v))
                cost += d['weight']
        return mst, cost

    def dijkstra(self, start, end): # [cite: 54, 88, 90]
        path = nx.shortest_path(self.G, start, end, weight='weight')
        dist = nx.shortest_path_length(self.G, start, end, weight='weight')
        return path, dist

        # --- PARTIE IV : Bellman-Ford & Floyd-Warshall ---
    def run_bellman_ford(self, start):
        """Détecte les cycles négatifs et calcule les distances [cite: 98-103]."""
        G_test = self.G.to_directed()
        # On ajoute un poids négatif contrôlé pour la démonstration [cite: 100]
        G_test.add_edge("Lyon", "Grenoble", weight=-50) 
        
        try:
            dist = nx.single_source_bellman_ford_path_length(G_test, start)
            return dist, "Aucun cycle négatif."
        except nx.NetworkXUnboundedCycle:
            return None, "Alerte : Cycle négatif détecté !"

    def run_floyd_warshall(self):
        """Calcule toutes les distances et identifie la ville centrale [cite: 105-111]."""
        matrix = nx.floyd_warshall_numpy(self.G, weight='weight')
        nodes = list(self.G.nodes())
        # Analyse : ville avec la distance moyenne la plus courte vers les autres [cite: 111]
        sums = np.sum(matrix, axis=1)
        central_idx = np.argmin(sums)
        return pd.DataFrame(matrix, index=nodes, columns=nodes), nodes[central_idx]

    # --- PARTIE V : Méthode PERT ---
    def run_pert(self):
        """Modélise l'ordonnancement d'un projet de route [cite: 112-122]."""
        # Tâches : (Nom, Durée, Dépendances) [cite: 116-118]
        tasks = {
            'A': (5, []),        # Étude de terrain
            'B': (10, ['A']),    # Terrassement
            'C': (8, ['A']),     # Fondations
            'D': (15, ['B', 'C']), # Bitume
            'E': (5, ['D'])      # Signalisation
        }
        
        dag = nx.DiGraph()
        for task, (dur, deps) in tasks.items():
            for dep in deps:
                dag.add_edge(dep, task, weight=tasks[dep][0])
        
        # Calcul du chemin critique (plus long chemin dans un DAG) [cite: 127]
        critical_path = nx.dag_longest_path(dag, weight='weight')
        total_duration = nx.dag_longest_path_length(dag, weight='weight') + tasks[critical_path[-1]][0]
        return tasks, critical_path, total_duration

    # (Garder ici les méthodes BFS, DFS, Kruskal, Dijkstra précédentes)
    def tracer_graphe(self, titre, path=None, edges=None):
        plt.figure(figsize=(8, 5))
        nx.draw(self.G, self.pos, with_labels=True, node_color='#A0CBE2', edge_color='gray', node_size=1200, font_size=8)
        if path:
            p_edges = [(path[i], path[i+1]) for i in range(len(path)-1)]
            nx.draw_networkx_edges(self.G, self.pos, edgelist=p_edges, edge_color='red', width=3)
        if edges:
            nx.draw_networkx_edges(self.G, self.pos, edgelist=edges, edge_color='blue', width=3)
        plt.title(titre)
        plt.show()

In [13]:
# Initialisation avec ton fichier importé 
engine = GraphEngine('graphe.json')

def menu_interactif():
    while True:
        print("\n--- MENU PROJET ALMF51 ---")
        print("1. Parcours (BFS/DFS)")
        print("2. Arbre Couvrant (Kruskal/Prim)")
        print("3. Plus court chemin (Dijkstra)")
        print("4. Bellman-Ford (Cycles négatifs)")
        print("5. Floyd-Warshall (Matrice complète)")
        print("6. Méthode PERT (Chemin critique)")
        print("0. Quitter")
        
        choix = input("\nEntrez votre choix (1-5) : ")
        
        if choix == '1':
            ordre = engine.bfs("Rennes") # [cite: 70]
            print(f"Ordre BFS : {ordre}")
            engine.tracer_graphe(f"Parcours BFS depuis Rennes", path=ordre)
            
        elif choix == '2':
            ordre = engine.dfs("Rennes") # [cite: 70]
            print(f"Ordre DFS : {ordre}")
            engine.tracer_graphe(f"Parcours DFS depuis Rennes", path=ordre)
            
        elif choix == '3':
            mst, cout = engine.kruskal() # [cite: 78, 81]
            print(f"Arêtes du MST : {mst}\nCoût total : {cout}")
            engine.tracer_graphe(f"Arbre Couvrant Minimum (Coût: {cout})", edges=mst)
            
        elif choix == '4':
            dist, msg = engine.run_bellman_ford("Rennes")
            print(f"Résultat : {msg}\nDistances : {dist}")
            
        elif choix == '5':
            df, centrale = engine.run_floyd_warshall()
            print("\n--- Matrice des distances ---")
            print(df)
            print(f"\nVille la plus centrale : {centrale}")
            
        elif choix == '6':
            tasks, path, total = engine.run_pert()
            print(f"\nProjet routier : {list(tasks.keys())}")
            print(f"Chemin critique : {' -> '.join(path)}")
            print(f"Durée totale minimale : {total} jours")
                
        elif choix == '0':
            print("Fin du programme.")
            break
        else:
            print("Choix invalide.")

# Lancement de l'interface
menu_interactif()


--- MENU PROJET ALMF51 ---
1. Parcours (BFS/DFS)
2. Arbre Couvrant (Kruskal/Prim)
3. Plus court chemin (Dijkstra)
4. Bellman-Ford (Cycles négatifs)
5. Floyd-Warshall (Matrice complète)
6. Méthode PERT (Chemin critique)
0. Quitter



Entrez votre choix (1-5) :  0


Fin du programme.
